In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.preprocessing import OneHotEncoder,PolynomialFeatures, MaxAbsScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn import preprocessing
import dill
import math
from sklearn import base

In [3]:
from sklearn.base import BaseEstimator, RegressorMixin


In [4]:
from sklearn.utils import shuffle

In [5]:
from timeit import default_timer as timer


In [6]:
inpatients=pd.read_csv('prepared_data/Inpatient.csv')

In [7]:
local=pd.read_csv('prepared_data/Output_local_basis.csv')

In [8]:
hospitals=pd.read_csv('prepared_data/hospitals.csv')

In [9]:
temp=local.merge(hospitals, left_on='zipcode2017', right_on='zipcode')

In [10]:
full_inner=inpatients.merge(temp, left_on='prov_id', right_on='Facility ID')

In [11]:
categorical_columns = ['DRG','hrrnum','hosp_ownership','prov_id']
numeric_columns=['average_AGI_c', 'average_wage_c','per_over_65','per_ba','per_h','Score' ]

In [12]:
y = full_inner['average_covered_charges']
X = full_inner.loc[:,categorical_columns+ numeric_columns]

### Model 1: with hospital fixed effects


In [13]:
#Split the data


X, y = shuffle(X, y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)

In [14]:
categorical_columns_0 = ['DRG', 'prov_id']

In [154]:
start = timer()
features = ColumnTransformer([
    ('categorical', OneHotEncoder(handle_unknown='ignore'), categorical_columns_0)])

est_0 = Pipeline([
    ('features', features),
    ('estimator', LinearRegression())])

est_0.fit(X_train[categorical_columns_0], y_train)
end = timer()

print(end-start)
print(f'R^2 score on train data: {est_0.score(X_train[categorical_columns_0], y_train)}')
print(f'R^2 score on test data: {est_0.score(X_test[categorical_columns_0], y_test)}')

y_pred=est_0.predict(X_test[categorical_columns_0])
print('average error rate on test set: ', abs((y_pred-y_test)/y_test).mean())
print('average error  on test set: ', abs((y_pred-y_test)).mean())

0.5605525619976106
R^2 score on train data: 0.7981749042587953
R^2 score on test data: 0.7897874718265723
average error rate on test set:  0.33582241814272457
average error  on test set:  16058.940547323824


In [15]:
class ResidualEstimator(BaseEstimator, RegressorMixin):

    def __init__(self,n_estimators=100,min_samples_leaf=1000,max_features='auto'):
        self.n_estimators=n_estimators
        self.min_samples_leaf=min_samples_leaf
        self.max_features=max_features

    def fit(self, X, y):
        est_1=LinearRegression().fit(X, y)
        res = y-est_1.predict(X)
        est_2=RandomForestRegressor(min_samples_leaf=self.min_samples_leaf,
                                    max_features=self.max_features,
                                    n_estimators=self.n_estimators).fit(X, res)
        self.estimator_1_=est_1
        self.estimator_2_=est_2
        return self

    def predict(self, X):
        return self.estimator_1_.predict(X)+self.estimator_2_.predict(X)


In [16]:
categorical_columns_1 = ['DRG','hrrnum','hosp_ownership', 'prov_id']
numeric_columns_1=['average_AGI_c', 'average_wage_c','per_over_65','per_ba','per_h']

In [17]:
X_train.columns

Index(['DRG', 'hrrnum', 'hosp_ownership', 'prov_id', 'average_AGI_c',
       'average_wage_c', 'per_over_65', 'per_ba', 'per_h', 'Score'],
      dtype='object')

In [18]:
start = timer()
features = ColumnTransformer([
    ('categorical', OneHotEncoder(handle_unknown='ignore'), categorical_columns_1),
('numeric', 'passthrough', numeric_columns_1)])

est_1 = Pipeline([
    ('features', features),
    ('estimator', ResidualEstimator(min_samples_leaf=200,n_estimators=100,max_features="auto"))])

est_1.fit(X_train[categorical_columns_1+numeric_columns_1], y_train
         # , estimator__sample_weight=X_train['total_discharges']
         )
end = timer()

print(end-start)
print(f'R^2 score on train data: {est_1.score(X_train[categorical_columns_1+numeric_columns_1], y_train)}')
print(f'R^2 score on test data: {est_1.score(X_test[categorical_columns_1+numeric_columns_1], y_test)}')

y_pred=est_1.predict(X_test[categorical_columns_1+numeric_columns_1])
print('average error rate on test set: ', abs((y_pred-y_test)/y_test).mean())
print('average error  on test set: ', abs((y_pred-y_test)).mean())

311.67021627500003
R^2 score on train data: 0.8013936494337388
R^2 score on test data: 0.8062174151707805
average error rate on test set:  0.29635704019889314
average error  on test set:  15317.97822130314


In [162]:
#Fit the model on the whole dataset
est_1.fit(X[categorical_columns_1+numeric_columns_1], y)
print(f'R^2 score on whole data: {est_1.score(X[categorical_columns_1+numeric_columns_1], y)}')

R^2 score on whole data: 0.8043417887554274


In [163]:
#dill.dump(est_1,open('prepared_data/est_1.pkd', 'wb'))  

### Model 2: without hospital fixed effects but with score


In [19]:
full_inner_w_spending=full_inner[full_inner.Footnote.isna()]

In [20]:
y_2 = full_inner_w_spending['average_covered_charges']
X_2 = full_inner_w_spending.loc[:,categorical_columns+ numeric_columns]

In [21]:
X_2, y_2 = shuffle(X_2, y_2)
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_2, y_2, test_size=0.15)

In [22]:
categorical_columns_2 = ['DRG','hrrnum','hosp_ownership']
numeric_columns_2=[ 'average_wage_c','per_over_65','Score','average_AGI_c','per_ba','per_h']

In [23]:
start = timer()
features = ColumnTransformer([
    ('categorical', OneHotEncoder(handle_unknown='ignore'), categorical_columns_2),
('numeric', 'passthrough', numeric_columns_2)])

est_2 = Pipeline([
    ('features', features),
    ('estimator', ResidualEstimator(min_samples_leaf=150,n_estimators=100,max_features="auto"))])

est_2.fit(X_train_2[categorical_columns_2+numeric_columns_2], y_train_2
         # , estimator__sample_weight=X_train['total_discharges']
         )
end = timer()

print(end-start)
print(f'R^2 score on train data: {est_2.score(X_train_2[categorical_columns_2+numeric_columns_2], y_train_2)}')
print(f'R^2 score on test data: {est_2.score(X_test_2[categorical_columns_2+numeric_columns_2], y_test_2)}')

y_pred_2=est_2.predict(X_test_2[categorical_columns_2+numeric_columns_2])
print('average error rate on test set: ', abs((y_pred_2-y_test_2)/y_test_2).mean())
print('average error  on test set: ', abs((y_pred_2-y_test_2)).mean())

264.528245568
R^2 score on train data: 0.7793898860661477
R^2 score on test data: 0.7777234989339465
average error rate on test set:  0.3369869226458439
average error  on test set:  16961.374077198478


In [169]:
est_2.fit(X_2[categorical_columns_2+numeric_columns_2], y_2)

Pipeline(steps=[('features',
                 ColumnTransformer(transformers=[('categorical',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['DRG', 'hrrnum',
                                                   'hosp_ownership']),
                                                 ('numeric', 'passthrough',
                                                  ['average_wage_c',
                                                   'per_over_65', 'Score',
                                                   'average_AGI_c', 'per_ba',
                                                   'per_h'])])),
                ('estimator', ResidualEstimator(min_samples_leaf=150))])

In [170]:
#dill.dump(est_2,open('prepared_data/est_2.pkd', 'wb'))  

### Model 3: without hospital fixed effects and without score


In [24]:
categorical_columns_3 = ['DRG','hrrnum','hosp_ownership']
numeric_columns_3=[ 'average_wage_c','per_over_65','average_AGI_c','per_ba','per_h']

In [25]:
features = ColumnTransformer([
    ('categorical', OneHotEncoder(handle_unknown='ignore'), categorical_columns_3),
('numeric', 'passthrough', numeric_columns_3)])

est_3 = Pipeline([
    ('features', features),
    ('estimator', ResidualEstimator(min_samples_leaf=100,n_estimators=200,max_features="auto"))])

In [26]:
start = timer()

est_3.fit(X_train[categorical_columns_3+numeric_columns_3], y_train
         # , estimator__sample_weight=X_train['total_discharges']
         )
end = timer()

print(end-start)
print(f'R^2 score on train data: {est_3.score(X_train[categorical_columns_3+numeric_columns_3], y_train)}')
print(f'R^2 score on test data: {est_3.score(X_test[categorical_columns_3+numeric_columns_3], y_test)}')

y_pred=est_3.predict(X_test[categorical_columns_3+numeric_columns_3])
print('average error rate on test set: ', abs((y_pred-y_test)/y_test).mean())
print('average error  on test set: ', abs((y_pred-y_test)).mean())

640.1274309670002
R^2 score on train data: 0.7686562791482419
R^2 score on test data: 0.7755013871373022
average error rate on test set:  0.35497168699725357
average error  on test set:  17328.46852907645


In [188]:
est_3.fit(X[categorical_columns_3+numeric_columns_3], y)

Pipeline(steps=[('features',
                 ColumnTransformer(transformers=[('categorical',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['DRG', 'hrrnum',
                                                   'hosp_ownership']),
                                                 ('numeric', 'passthrough',
                                                  ['average_wage_c',
                                                   'per_over_65',
                                                   'average_AGI_c', 'per_ba',
                                                   'per_h'])])),
                ('estimator',
                 ResidualEstimator(min_samples_leaf=100, n_estimators=200))])

In [189]:
#dill.dump(est_3,open('prepared_data/est_3.pkd', 'wb'))  

### Model 4: combination of models 1 and 2

In [27]:
X_comb_test=np.concatenate((np.reshape(est_1.predict(X_test_2[categorical_columns_1 + numeric_columns_1]), (-1,1)),
                            np.reshape(est_2.predict(X_test_2[categorical_columns_2 + numeric_columns_2]), (-1,1))), axis=1)
X_comb_train=np.concatenate((np.reshape(est_1.predict(X_train_2[categorical_columns_1 + numeric_columns_1]), (-1,1)),
                             np.reshape(est_2.predict(X_train_2[categorical_columns_2 + numeric_columns_2]), (-1,1))), axis=1)


In [191]:
est_4 = RandomForestRegressor(max_depth=6, random_state=0)
est_4.fit(X_comb_train, y_train_2)

print('average error rate on the training set: ',abs((est_4.predict(X_comb_train)-y_train_2)/y_train_2).mean())
print('average error on the training set: ',abs((est_4.predict(X_comb_train)-y_train_2)).mean())

print(f'R^2 score using selected columns and transformers: {est_4.score(X_comb_test, y_test_2)}')

print('average error rate on the test set: ',abs((est_4.predict(X_comb_test)-y_test_2)/y_test_2).mean())
print('average error  on the test set: ',abs((est_4.predict(X_comb_test)-y_test_2)).mean())


average error rate on the training set:  0.212563418032397
average error on the training set:  12678.137187007032
R^2 score using selected columns and transformers: 0.8748432393262726
average error rate on the test set:  0.21287581962549526
average error  on the test set:  12737.332542758293


In [192]:
dill.dump(est_4,open('prepared_data/est_4.pkd', 'wb'))  

In [126]:
%%bash
ls

Models.ipynb
Visualizations.ipynb
prepared_data
raw_inputs


In [133]:
%%bash
git commit -m 'add the app backbone, including the html pages, the app.py, the css file, and Dockerfile but no data'

[master 4020101] add the app backbone, including the html pages, the app.py, the css file, and Dockerfile but no data
 14 files changed, 1001 insertions(+)
 create mode 100644 Dockerfile
 create mode 100644 app/.DS_Store
 create mode 100644 app/app.py
 create mode 100644 app/conda-requirements.txt
 create mode 100644 app/requirements.txt
 create mode 100644 app/static/.DS_Store
 create mode 100644 app/static/style.css
 create mode 100644 app/templates/.DS_Store
 create mode 100644 app/templates/Model.html
 create mode 100644 app/templates/about2.html
 create mode 100644 app/templates/index.html
 create mode 100644 app/templates/index_2.html
 create mode 100644 app/templates/index_3.html
 create mode 100644 app/templates/results.html


## Save the list of variables for the app and the dataset used

In [128]:
dill.dump((categorical_columns,numeric_columns,
            categorical_columns_1,numeric_columns_1,
           categorical_columns_2,numeric_columns_2,
           categorical_columns_3,numeric_columns_3), open('prepared_data/cat_num_col.pkd', 'wb'))  


**Prepare the dataset for the app**

In [28]:
temp=local.merge(hospitals, left_on='zipcode2017', right_on='zipcode')

In [29]:
#Let's incorporate the data from inpatient, just the information present or not

inpatient_list_hosp=inpatients[['prov_id','prov_zip']].drop_duplicates()
inpatient_list_hosp['Inpatients_avail']=1
inpatient_list_hosp=inpatient_list_hosp[['prov_id','Inpatients_avail']]

In [30]:
temp_2=inpatient_list_hosp.merge(temp, how='right',left_on='prov_id',right_on='Facility ID')

In [31]:
temp_2['Score_avail']=temp_2.Score.apply(lambda x : 1 if x != 'Not Available' else 0)

In [32]:
temp_2['Score']=temp_2.Score.apply(lambda x : float(x) if x != 'Not Available' else None)

In [33]:
temp_2['Inpatients_avail']=temp_2['Inpatients_avail'].fillna(value=0)

In [34]:
temp_2['hosp_category']=0

In [35]:
temp_2.loc[(temp_2.Inpatients_avail==1)&(temp_2.Score_avail==0),'hosp_category']=1
temp_2.loc[(temp_2.Inpatients_avail==1)&(temp_2.Score_avail==1),'hosp_category']=4
temp_2.loc[(temp_2.Inpatients_avail==0)&(temp_2.Score_avail==0),'hosp_category']=3
temp_2.loc[(temp_2.Inpatients_avail==0)&(temp_2.Score_avail==1),'hosp_category']=2

In [36]:
temp_2.hosp_category.value_counts()

4    2985
3      85
1      59
2      39
Name: hosp_category, dtype: int64

In [318]:
temp_2.rename(columns={'City':'countyname',
                    'Facility ID' : 'Provider ID'},
           inplace=True)



In [320]:
temp_2['prov_id']=temp_2['Provider ID']
temp_2.describe()

,prov_id,Inpatients_avail,zipcode2017,hrrnum,county,Latitude,Longitude,average_AGI_c,average_wage_c,COUNTY_ID,per_over_65,per_ba,per_h,Provider ID,Score,Footnote,zipcode,hosp_rating_fn,Score_avail,hosp_category
count,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3024.000000,144.000000,3168.000000,330.000000,3168.000000,3168.000000
mean,262218.210859,0.960859,52093.619003,249.690657,28417.768308,37.312330,-91.799018,66.638548,55.641617,28417.768308,0.161957,0.131522,0.154186,262218.210859,0.986121,6.041667,52093.619003,16.054545,0.954545,3.892677
std,161909.590674,0.193962,27818.339780,126.757860,15989.995363,5.094979,15.090129,23.910746,16.767326,15989.995363,0.040010,0.137957,0.164017,161909.590674,0.080031,5.936582,27818.339780,2.205973,0.208332,0.481915
min,10001.000000,0.000000,1040.000000,1.000000,1001.000000,19.526845,-161.880060,4.811600,27.336667,1001.000000,0.047665,0.000831,0.006673,10001.000000,0.480000,1.000000,1040.000000,5.000000,0.000000,1.000000
25%,110114.500000,1.000000,30119.250000,144.000000,13154.500000,33.680992,-97.508440,51.639582,44.518386,13154.500000,0.134672,0.028352,0.038820,110114.500000,0.940000,1.000000,30119.250000,16.000000,1.000000,4.000000
50%,250084.500000,1.000000,48840.500000,268.000000,28100.000000,37.766329,-87.875455,61.173231,51.938428,28100.000000,0.157918,0.085894,0.087132,250084.500000,0.990000,5.000000,48840.500000,16.000000,1.000000,4.000000
75%,390120.000000,1.000000,76241.000000,357.000000,42056.500000,41.070054,-81.175125,76.795411,62.346322,42056.500000,0.182523,0.191344,0.223079,390120.000000,1.030000,5.000000,76241.000000,16.000000,1.000000,4.000000
max,670130.000000,1.000000,99801.000000,457.000000,56041.000000,64.835070,-68.002160,264.815267,146.194817,56041.000000,0.569440,0.854172,0.963230,670130.000000,1.530000,19.000000,99801.000000,23.000000,1.000000,4.000000


In [321]:
#['Provider ID', 'DRG', 'prov_id', 'total_discharges', 'countyname']

X_full=temp_2[['hosp_name','zipcode','state','Latitude','Longitude','Provider ID','countyname','hosp_rating' ]
                                      +numeric_columns +['hrrnum', 'hosp_ownership', 'prov_id','hosp_category']]
X_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3168 entries, 0 to 3167
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   hosp_name       3168 non-null   object 
 1   zipcode         3168 non-null   int64  
 2   state           3168 non-null   object 
 3   Latitude        3168 non-null   float64
 4   Longitude       3168 non-null   float64
 5   Provider ID     3168 non-null   int64  
 6   countyname      3168 non-null   object 
 7   hosp_rating     3168 non-null   object 
 8   average_AGI_c   3168 non-null   float64
 9   average_wage_c  3168 non-null   float64
 10  per_over_65     3168 non-null   float64
 11  per_ba          3168 non-null   float64
 12  per_h           3168 non-null   float64
 13  Score           3024 non-null   float64
 14  hrrnum          3168 non-null   int64  
 15  hosp_ownership  3168 non-null   object 
 16  prov_id         3168 non-null   int64  
 17  hosp_category   3168 non-null   i

In [322]:
dill.dump(X_full, open('prepared_data/X_full.pkd', 'wb'))  

In [323]:
X_full['price']=0

<ipython-input-323-8a92de76533e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_full['price']=0


In [324]:
X_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3168 entries, 0 to 3167
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   hosp_name       3168 non-null   object 
 1   zipcode         3168 non-null   int64  
 2   state           3168 non-null   object 
 3   Latitude        3168 non-null   float64
 4   Longitude       3168 non-null   float64
 5   Provider ID     3168 non-null   int64  
 6   countyname      3168 non-null   object 
 7   hosp_rating     3168 non-null   object 
 8   average_AGI_c   3168 non-null   float64
 9   average_wage_c  3168 non-null   float64
 10  per_over_65     3168 non-null   float64
 11  per_ba          3168 non-null   float64
 12  per_h           3168 non-null   float64
 13  Score           3024 non-null   float64
 14  hrrnum          3168 non-null   int64  
 15  hosp_ownership  3168 non-null   object 
 16  prov_id         3168 non-null   int64  
 17  hosp_category   3168 non-null   i

In [325]:
categorical_columns_1

['DRG', 'hrrnum', 'hosp_ownership', 'prov_id']

In [326]:
X_full.loc[:,'DRG']='023 - CRANIOTOMY W MAJOR DEVICE IMPLANT OR ACUTE CNS PDX W MCC OR CHEMOTHE'

/Users/camilledethe/opt/anaconda3/envs/cap_env/lib/python3.8/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/camilledethe/opt/anaconda3/envs/cap_env/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [327]:
X_full.hosp_category.describe()

count    3168.000000
mean        3.892677
std         0.481915
min         1.000000
25%         4.000000
50%         4.000000
75%         4.000000
max         4.000000
Name: hosp_category, dtype: float64

In [339]:
est_1.predict(X_full.loc[X_full.hosp_category==1,categorical_columns_1+ numeric_columns_1])
                                                                  

array([196009.61933385, 177318.98540027, 170700.01327134, 427576.42994604,
       152150.47238619, 144305.00672189, 158260.07911253, 150585.41661411,
       155764.46154518, 164200.76954573, 147072.2044009 , 152292.06846962,
       154963.54565179, 149887.26716402, 173224.1880926 , 157142.6205372 ,
       146118.3703418 , 158403.89268349, 162436.73450087, 142114.39665409,
       147529.40190455, 148553.5745423 , 146509.84925857, 152995.85866158,
       160372.96136325, 155604.96379066, 166133.54583631, 161445.33366381,
       159927.29242297, 161748.37050682, 158698.18174595, 162775.00875258,
       163844.50716089, 161280.83371833, 162993.52680591, 147920.02645941,
       161624.19538449, 169190.71569187, 157465.34411397, 149690.36095143,
       154994.90645389, 162558.78903012, 160539.45790511, 152847.54048688,
       172491.44370084, 163804.55090402, 153373.08753666, 157680.08120402,
       151117.11319253, 196974.50717297, 160393.70038602, 183997.78513604,
       170129.95307975, 1

In [344]:
X_full.loc[X_full.hosp_category==1,'price']=est_1.predict(X_full.loc[X_full.hosp_category==1,categorical_columns_1+ numeric_columns_1])
X_full.loc[X_full.hosp_category==2,'price']=est_2.predict(X_full.loc[X_full.hosp_category==2,categorical_columns_2+ numeric_columns_2])
X_full.loc[X_full.hosp_category==3,'price']=est_3.predict(X_full.loc[X_full.hosp_category==3,categorical_columns_3+ numeric_columns_3])

X_1_2=np.concatenate((np.reshape(est_1.predict(X_full.loc[X_full.hosp_category==4,categorical_columns_1+ numeric_columns_1]), (-1,1)),
                            np.reshape(est_2.predict(X_full.loc[X_full.hosp_category==4,categorical_columns_2+ numeric_columns_2]), (-1,1))), axis=1)


X_full.loc[X_full.hosp_category==4,'price']=est_4.predict(X_1_2)


/Users/camilledethe/opt/anaconda3/envs/cap_env/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/Users/camilledethe/opt/anaconda3/envs/cap_env/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/Users/camilledethe/opt/anaconda3/envs/cap_env/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sl

In [346]:
X_full.describe()

,zipcode,Latitude,Longitude,Provider ID,average_AGI_c,average_wage_c,per_over_65,per_ba,per_h,Score,hrrnum,prov_id,hosp_category,price
count,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3024.000000,3168.000000,3168.000000,3168.000000,3168.000000
mean,52093.619003,37.312330,-91.799018,262218.210859,66.638548,55.641617,0.161957,0.131522,0.154186,0.986121,249.690657,262218.210859,3.892677,191025.761971
std,27818.339780,5.094979,15.090129,161909.590674,23.910746,16.767326,0.040010,0.137957,0.164017,0.080031,126.757860,161909.590674,0.481915,36302.914636
min,1040.000000,19.526845,-161.880060,10001.000000,4.811600,27.336667,0.047665,0.000831,0.006673,0.480000,1.000000,10001.000000,1.000000,101870.242366
25%,30119.250000,33.680992,-97.508440,110114.500000,51.639582,44.518386,0.134672,0.028352,0.038820,0.940000,144.000000,110114.500000,4.000000,172464.562058
50%,48840.500000,37.766329,-87.875455,250084.500000,61.173231,51.938428,0.157918,0.085894,0.087132,0.990000,268.000000,250084.500000,4.000000,180206.832967
75%,76241.000000,41.070054,-81.175125,390120.000000,76.795411,62.346322,0.182523,0.191344,0.223079,1.030000,357.000000,390120.000000,4.000000,205068.743273
max,99801.000000,64.835070,-68.002160,670130.000000,264.815267,146.194817,0.569440,0.854172,0.963230,1.530000,457.000000,670130.000000,4.000000,529609.756725


In [ ]:
X_full['price'].describe()

In [284]:
inpatients.iloc[0,0]

'023 - CRANIOTOMY W MAJOR DEVICE IMPLANT OR ACUTE CNS PDX W MCC OR CHEMOTHE'